##Installs

In [ ]:
#!pip install scikit-learn
from os import system
# system("pip3 install essentia")
# system("pip3 install scikit-learn")

##DEFS

In [ ]:
#If not defined then define
def ifNDef(varname=None, value=None):
  assert varname
  assert value
  if not varname in globals():
    globals()[varname] = value

ifNDef("PATH_NOTEBOOKS", "../notebooks/")

ifNDef("PATH_AUDIO", "../audio/")

ifNDef("PATH_RITMOS", PATH_AUDIO+"ritmos/")

ifNDef("PATH_DATAPOINTS", PATH_AUDIOS+"dataset/datapoints/")

ifNDef("AUDIO_FILES", ["Mono1.WAV", "Mono2.WAV", "Mono3.WAV", "Mono4.WAV", "Stereo1.WAV"])

ifNDef("essentia_mode", "standard")

##Imports

In [ ]:
from os import remove
from google.colab import files

import json


import numpy as np
print("numpy version: ", np.__version__)

import matplotlib.pyplot as plt

# first, we need to import our essentia module. It is aptly named 'essentia'!
import essentia as es
print("essentia version: ",es.__version__)

if essentia_mode.lower() == "streaming":
  import essentia.streaming as es_mode
else:
  import essentia.standard as es_mode

#Imports para setup
from IPython.display import Audio
from pylab import plot, show, figure, imshow
from os import listdir, path


numpy version:  1.18.3
essentia version:  2.1-beta6-dev


##Definições

###Upload dos arquivos de áudio de teste

In [ ]:
def checkSampleRate(sr):
  return 44100

def load_audio_from_project(audios_path="../audio/preliminares/",
                            audio_name=None,
                            print_output=True):
  """
  Carrega os áudios em audios_path. Se audio_name estiver definido, carrega-o
  """

  output = {}
  if not audio_name:
    audio_files = [f for f in listdir(audios_path) if not path.isdir(path.join(audios_path,f))]
    for name in audio_files:
      metadata = es_mode.MetadataReader(filename=audios_path+name)()
      loader = es_mode.MonoLoader(filename=(audios_path+name),\
                                  sampleRate=(checkSampleRate(metadata[10])))
      if print_output: print("Loaded", name, "with sampleRate:", checkSampleRate(metadata[10]))
      output[name] = {"data":loader(), 
                      "path":audios_path+name,
                      "sample_rate": checkSampleRate(metadata[10]),
                      "metadata":metadata}
  else:
    metadata = es_mode.MetadataReader(filename=audios_path+audio_name)()
    loader = es_mode.MonoLoader(filename=(audios_path+audio_name),\
                                sampleRate=checkSampleRate(metadata[10]))
    if print_output: print("Loaded", audio_name, "with sampleRate:", checkSampleRate(metadata[10]))
    output[audio_name] = {"data":loader(), 
                          "path":audios_path+audio_name,
                          "sample_rate": checkSampleRate(metadata[10]),
                          "metadata":metadata}

  return output


In [ ]:
hear_audio = Audio

In [ ]:
def see_audio(audio, aux_plot=None):
  plt.rcParams['figure.figsize'] = (15, 6) # set plot sizes to something larger than default
  plot(audio)
  if aux_plot:
    aux_plot()
  show()

In [ ]:
#use pool dentro do algoritmo para salvar as features
def inFrames(audio, algorithm, frameSize=1024, hopSize=512,
             lastFrameToEndOfFile=False,
             startFromZero=False,
             validFrameThresholdRatio=0):
  from essentia.standard import FrameGenerator
  qtdFrames = 0
  for frame in es_mode.FrameGenerator(audio=audio, 
                                      frameSize=frameSize,
                                      hopSize=hopSize,
                                      lastFrameToEndOfFile=lastFrameToEndOfFile,
                                      startFromZero=startFromZero,
                                      validFrameThresholdRatio=validFrameThresholdRatio):
    algorithm(frame)
    qtdFrames += 1
  return qtdFrames